In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
import datetime as dt
from datetime import datetime
import bz2
import pickle
import _pickle as cPickle

%matplotlib inline

In [32]:
tickers = pd.read_csv('russell_1.csv')

In [ ]:
yf.

In [33]:
tickers

,Symbol,Name,Last,Change,%Chg,Open,High,Low,Volume,Time
0,A,Agilent Technologies,96.84,-2.80,-2.81%,100.07,100.11,95.66,1753874.0,09/04/20
1,AA,Alcoa Corp,14.40,0.17,+1.19%,14.52,14.67,13.90,3929632.0,09/04/20
2,AAL,American Airlines Gp,13.61,0.25,+1.87%,13.65,13.82,12.96,64936898.0,09/04/20
3,AAN,Aaron's Inc,54.67,-0.96,-1.73%,56.46,56.54,52.99,472870.0,09/04/20
4,AAOI,Applied Optoelect,10.52,-0.43,-3.93%,11.00,11.07,9.88,948500.0,09/04/20
...,...,...,...,...,...,...,...,...,...,...
996,FMAO,Farmers & Merch Banc,21.83,-0.27,-1.22%,22.25,22.25,21.56,6800.0,09/04/20
997,FMBH,First Mid Ill Bncshr,26.38,0.51,+1.97%,26.37,26.66,25.99,16900.0,09/04/20
998,FMBI,First Midwest Bncp,12.82,0.26,+2.11%,12.92,13.03,12.63,415300.0,09/04/20
999,FMC,FMC Corp,108.48,-0.95,-0.87%,110.53,110.90,106.98,683211.0,09/04/20


In [34]:
tickers = tickers['Symbol'].to_list()

In [ ]:
#tickers = ['FB', 'AA', 'AAPL']
stocks = download_tickers(tickers[600:900])

[*********************100%***********************]  250 of 250 completed

1 Failed download:
- BF.A: No data found, symbol may be delisted


In [163]:
stocks.loc[:, stocks.columns.get_level_values(1).isin(['Adj Close'])].iloc[:,3:11]

Stock ticker,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AFL
Data type,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close
Date,,,,,,,,
1962-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-08-31,307.619995,239.929993,83.519997,513.390015,90.820000,156.309998,17.750000,36.320000
2020-09-01,307.329987,240.020004,83.330002,527.950012,92.180000,156.389999,17.910000,36.700001


In [4]:
def download_tickers(tickers):
    # Fetch downloaded ticker from raw_data
    downloaded_tickers = [x[0:-5] if '.pbz2' in x else '' for x in os.listdir('raw_data/')]
    downloaded_tickers.remove('')
    # Check if tickers already are downloaded
    tickers_to_download = []
    for ticker in tickers:
        if ticker in downloaded_tickers:
            # If already downloaded, check if they need updates
            if update_ticker(ticker):
                tickers_to_download.append(ticker)
        else:
            tickers_to_download.append(ticker)

    download_dump(tickers_to_download)
    
    # Read tickers from json and return
    return load_stocks(tickers)

In [5]:
def load_stocks(tickers):
    cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    index = pd.MultiIndex.from_product([tickers, cols], names=['Stock ticker', 'Data type'])
    data = pd.concat([decompress_pickle('raw_data/' + x + '.pbz2') for x in tickers], axis=1, sort=True)
    data.columns = index
    try:
        data.index = pd.to_datetime(data.index, unit='ms')
    except ValueError:
        data.index = pd.to_datetime(data.index)
    return data

In [6]:
def update_ticker(ticker):
    last_bday = last_weekday()
    newest_date = decompress_pickle('raw_data/' + ticker + '.pbz2').index[-1]
    if newest_date == float:
        newest_date = dt.datetime.fromtimestamp(newest_date/1000).date()
    if newest_date != last_bday:
        return True 
    else:
        return False

In [7]:
def download_dump(tickers):
    if tickers != []:
        data = yf.download(tickers, period='100y', group_by='tickers')
        if len(tickers) > 1:
            for ticker in tickers:
                    clean_data = clean_df(data[ticker])
                    #clean_data.to_json('raw_data/' + ticker + '.json', date_format='epoch', date_unit='ms')
                    compressed_pickle(ticker, data[ticker])
        else:
            clean_data = clean_df(data)
            #clean_data.to_json('raw_data/' + tickers[0] + '.json', date_format='epoch', date_unit='ms')
            compressed_pickle(ticker, data)

In [8]:
def last_weekday():
    todays_day = dt.date.today().day
    if todays_day in [5, 6]:
        return dt.date.today() - dt.timedelta(days=todays_day-4)
    else:
        return dt.date.today()


In [9]:
def clean_df(df):
    before = len(df)
    df = df.dropna(axis = 0, how='all')
    return df

In [10]:
def compressed_pickle(ticker, data):
    with bz2.BZ2File('raw_data/' + ticker + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [11]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [12]:
test = decompress_pickle('raw_data/AAPL.pbz2')

In [13]:
test

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.101261,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095978,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088934,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.091135,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093777,73449600
...,...,...,...,...,...,...
2020-08-31,127.580002,131.000000,126.000000,129.039993,129.039993,225702700
2020-09-01,132.759995,134.800003,130.529999,134.179993,134.179993,152470100
2020-09-02,137.589996,137.979996,127.000000,131.399994,131.399994,200119000
